In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
import requests
import datetime as dt
from scipy.stats import gompertz
from dateutil.relativedelta import *

# Create Bar Chart Race

In [5]:
df = pd.read_csv('./data_scrape.csv')
df.head()

,Tên kênh,Số lượt thích,Số lượt xem,Số bình luận,Tiêu đề,Ngày phát hành
0,Wowy,418.0,16519.0,10.0,(VER JBEE7) NGƯỜI VS WOWY - DUET CHALLENGE #n...,2023-11-01
1,Wowy,3772.0,163201.0,230.0,WOWY - NGƯỜI (M/V),2023-11-01
2,Wowy,85.0,2855.0,3.0,11.11 - MV NGƯỜI - WOWY TEASER,2023-11-01
3,Wowy,256.0,5893.0,5.0,Lời cảm ơn của #Wowy 🙏 sau khi bản #rap #Người...,2023-11-01
4,Wowy,201.0,6795.0,7.0,WOWY DIỄN LIVE CA KHÚC MỚI ĐẦY CHIÊM NGHIỆM #w...,2023-11-01


In [6]:
# Convert date
df['Ngày phát hành'] = df['Ngày phát hành'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

In [15]:
channel_names = []
likes = []
views = []
title = []
date = []
c = 5
month_now = 1
year_now = 2025

for _, row in df.iterrows():
    diff_month = 12 * year_now + month_now - 12 * row['Ngày phát hành'].year - row['Ngày phát hành'].month
    if diff_month < 1:
        print(f"Skipping row with invalid date: {row['Ngày phát hành']}")
        continue
    ratios = gompertz(c).cdf(np.linspace(gompertz.ppf(0.01, c),
                                         gompertz.ppf(0.99, c), diff_month + 1))[1:]
    ratios[-1] = 1

    for i, ratio in enumerate(ratios):
        channel_names.append(row['Tên kênh'])
        likes.append(row['Số lượt thích'] * ratio)
        views.append(row['Số lượt xem'] * ratio)
        title.append(row['Tiêu đề'])
        date.append(row['Ngày phát hành'] + relativedelta(months=i+1))

new_df = pd.DataFrame(data={'Tên kênh': channel_names,
                            'Số lượt thích': likes,
                            'Số lượt xem': views,
                            'Tiêu đề': title,
                            'Ngày phát hành': date})

In [27]:
# Tỉ lệ yêu thích (Like Ratio) = (Likes / Views) * 100%
# Tỉ lệ yêu thích bình quân (Normalized Like Ratio) = (Like Ratio / Like Ratio trung bình của kênh) * 100%
new_df = new_df[new_df['Số lượt xem'] > 0]
new_df['Tỉ lệ yêu thích'] = (new_df["Số lượt thích"] / new_df["Số lượt xem"])
create_feature_dict = {'Tỉ lệ yêu thích': 'mean',
                           'Số lượt xem': 'sum',
                           'Số lượt thích': 'sum'}
runbar_df = new_df.groupby(['Tên kênh', 'Ngày phát hành']).agg(create_feature_dict).reset_index()
runbar_df['Tỉ lệ yêu thích bình quân'] = (runbar_df['Tỉ lệ yêu thích'] / runbar_df['Tỉ lệ yêu thích'].mean())
runbar_df['Ngày phát hành'] = runbar_df['Ngày phát hành'].dt.to_period('M')
runbar_df['Tỉ lệ không yêu thích bình quân'] = 1 - runbar_df['Tỉ lệ yêu thích bình quân']
runbar_df

,Tên kênh,Ngày phát hành,Tỉ lệ yêu thích,Số lượt xem,Số lượt thích,Tỉ lệ yêu thích bình quân,Tỉ lệ không yêu thích bình quân
0,AMEE,2020-05,0.007130,5.301132e+05,3.536536e+03,0.494133,0.505867
1,AMEE,2020-06,0.018693,1.001354e+06,7.587714e+03,1.295413,-0.295413
2,AMEE,2020-07,0.019908,1.569716e+06,1.417538e+04,1.379568,-0.379568
3,AMEE,2020-08,0.022787,2.135380e+06,2.130540e+04,1.579118,-0.579118
4,AMEE,2020-09,0.023275,2.682727e+06,2.837211e+04,1.612923,-0.612923
...,...,...,...,...,...,...,...
2301,Đàm Vĩnh Hưng,2024-09,0.015751,8.058564e+08,4.742332e+06,1.091519,-0.091519
2302,Đàm Vĩnh Hưng,2024-10,0.015765,8.097184e+08,4.784772e+06,1.092521,-0.092521
2303,Đàm Vĩnh Hưng,2024-11,0.015765,8.124401e+08,4.813032e+06,1.092521,-0.092521
2304,Đàm Vĩnh Hưng,2024-12,0.015765,8.143860e+08,4.831898e+06,1.092521,-0.092521


### Bar Chart Race - Lượt xem

In [31]:
pivot_view = pd.pivot_table(runbar_df, values='Số lượt xem', index=['Tên kênh'],
                    columns=['Ngày phát hành']).reset_index()
pivot_view.head()

Ngày phát hành,Tên kênh,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,...,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01
0,AMEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.523357e+07,1.527767e+07,1.531767e+07,1.535391e+07,1.538670e+07,1.545952e+07,1.550929e+07,1.554472e+07,1.557138e+07,1.575030e+07
1,BÍCH PHƯƠNG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.138777e+09,1.146480e+09,1.152565e+09,1.157350e+09,1.161109e+09,1.164075e+09,1.166436e+09,1.168343e+09,1.169912e+09,1.183059e+09
2,Chi Pu Official,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.629416e+08,4.648444e+08,4.693782e+08,4.727829e+08,4.753640e+08,4.772892e+08,4.787126e+08,4.803281e+08,4.813200e+08,4.868615e+08
3,ERIK Official,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.650232e+08,7.679262e+08,7.703855e+08,7.725039e+08,7.743032e+08,7.758413e+08,7.772761e+08,7.786162e+08,7.797266e+08,7.885225e+08
4,Hoàng Thùy Linh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.624101e+08,6.653986e+08,6.679888e+08,6.702318e+08,6.721733e+08,6.738536e+08,6.753081e+08,6.765727e+08,6.779630e+08,6.858158e+08


In [29]:
pivot_view.to_csv('./pivot_view.csv', index=False)

### Bar Chart Race - Tỉ lệ yêu thích bình quân

In [32]:
pivot_favorite = pd.pivot_table(runbar_df, values='Tỉ lệ yêu thích bình quân', index=['Tên kênh'],
                    columns=['Ngày phát hành']).reset_index()
pivot_favorite.head()

Ngày phát hành,Tên kênh,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,...,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01
0,AMEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.666094,1.666094,1.666094,1.666094,1.666094,1.846271,1.846271,1.846271,1.846271,1.846271
1,BÍCH PHƯƠNG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.331314,1.344233,1.344233,1.344233,1.344233,1.344233,1.344233,1.344233,1.344233,1.344233
2,Chi Pu Official,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.244722,1.254822,1.307895,1.310608,1.317948,1.340235,1.343639,1.563023,1.606303,1.606303
3,ERIK Official,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.295713,1.295713,1.295713,1.311041,1.311041,1.311041,1.332116,1.354685,1.425498,1.425498
4,Hoàng Thùy Linh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.870374,0.870374,0.870374,0.870374,0.870374,0.870374,0.870374,0.870374,1.128875,1.128875


In [33]:
pivot_favorite.to_csv('./pivot_favorite.csv', index=False)

# Bar Chart Race - Lượng người theo dõi

In [53]:
channel_df = pd.read_csv('./channel_stats.csv')
channel_df['Ngày thành lập'] = pd.to_datetime(channel_df['Ngày thành lập'])
channel_df

,Tên kênh,Số lượt theo dõi,Ngày thành lập
0,Wowy,1230000,2012-03-01
1,JustaTeeMusic,1350000,2013-02-01
2,Đàm Vĩnh Hưng,1860000,2013-04-01
3,Trúc Nhân,1810000,2013-05-01
4,Hoàng Thùy Linh,1530000,2013-03-01
5,BÍCH PHƯƠNG,2340000,2014-11-01
6,My Tam,2450000,2013-11-01
7,Sơn Tùng M-TP Official,11200000,2015-01-01
8,Hòa Minzy,1770000,2014-10-01
9,MIN OFFICIAL,2090000,2015-12-01


In [54]:
channel_names = []
subscribers = []
months_years = []
c = 5
month_now = 1
year_now = 2025

for _, row in channel_df.iterrows():
    # Tính số tháng từ ngày thành lập đến thời điểm hiện tại (hoặc thời gian dự đoán)
    diff_month = 12 * year_now + month_now - 12 * row['Ngày thành lập'].year - row['Ngày thành lập'].month
    
    if diff_month < 1:
        print(f"Skipping row with invalid date: {row['Ngày thành lập']}")
        continue
    
    # Tính toán tỷ lệ tăng trưởng dựa trên mô hình Gompertz
    ratios = gompertz(c).cdf(np.linspace(gompertz.ppf(0.01, c),
                                         gompertz.ppf(0.99, c), diff_month + 1))[1:]
    ratios[-1] = 1  # Đảm bảo tỷ lệ cuối cùng là 1 (tối đa)

    # Tạo các giá trị cho từng tháng
    for i, ratio in enumerate(ratios):
        channel_names.append(row['Tên kênh'])
        subscribers.append(row['Số lượt theo dõi'] * ratio)  # ước tính lượt theo dõi
        month_year = (row['Ngày thành lập'] + relativedelta(months=i+1)).strftime('%Y-%m')
        months_years.append(month_year)


# Tạo DataFrame mới chứa kết quả
newChannel_df = pd.DataFrame({
    'Tên kênh': channel_names,
    'Số lượt theo dõi': subscribers,
    'Ngày thành lập': months_years
})
newChannel_df

,Tên kênh,Số lượt theo dõi,Ngày thành lập
0,Wowy,37866.122062,2012-04
1,Wowy,63000.353531,2012-05
2,Wowy,87707.765191,2012-06
3,Wowy,111993.408672,2012-07
4,Wowy,135862.315864,2012-08
...,...,...,...
2301,AMEE,156558.595956,2024-09
2302,AMEE,156802.780499,2024-10
2303,AMEE,157024.933819,2024-11
2304,AMEE,157226.797857,2024-12


In [55]:
pivot_subscriber = pd.pivot_table(newChannel_df, 
                                  values='Số lượt theo dõi', 
                                  index=['Tên kênh'], 
                                  columns=['Ngày thành lập'], 
                                  aggfunc='sum',  # You can adjust the aggregation function as needed (e.g., 'sum', 'mean')
                                  fill_value=0)   # Optional: Fill missing values with 0 or another value

pivot_subscriber = pivot_subscriber.reset_index()  # Reset index to make 'Tên kênh' a column
pivot_subscriber.to_csv('pivot_subscriber.csv', index = False)
